# 4bit quantization으로 Polyglot-ko 12.8B QLoRA 학습하기

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

- Colab Free (T4 GPU)에서, 한국어 언어 모델 중 가장 큰 크기 모델인 Polyglot-ko 12.8B 모델을 QLoRA로 파인튜닝 해 봅시다.

In [1]:
# nvidia-smi는 GPU 메모리 사용량, GPU 온도, 전력 소비, GPU 사용률 등의 정보를 실시간으로 표시하여 GPU 성능 및 리소스 사용에 대한 정보를 제공합니다.
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 MB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 데이터셋: KoAlpaca v1.1a

In [3]:
from datasets import load_dataset

# 학습데이터 들고 오기
data = load_dataset("beomi/KoAlpaca-v1.1a")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/beomi___parquet/beomi--KoAlpaca-v1.1a-1465f66eb846fd61/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
data
train_dataset = data["train"]

example_array = []
for i, example in enumerate(train_dataset[:10]):
  example = train_dataset[i]
  example_array.append(example)
  print(f"Example {i+1}:")
  for key, value in example.items():
    print(f"{key}: {value}")
  print()

print(example_array)

Example 1:
instruction: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?
output: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 

식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.

 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 

고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.
url: https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268

Example 2:
instruction: 스웨터의 유래는 어디에서 시작되었나요?
output: 스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 시작되었습니다. 노동자들의 방한복에서 시작된 스웨터는 여가생활과 스포츠의 붐에 힘입어 대중화되었습니다. 이후, 겨울철 이너웨어의 대명사가 되었습니다. 스웨터는 짜서(Knit) 만든 옷을 말하며, 어부들의 방한복으로 짜여졌던 스웨터 중에서도 스코틀랜드 해안지방의 여인들은 바다로 나가는 남편이나 연인, 자식들에게 무사히 돌아올 것을 기원하며 로프나 닻 무늬를 정성껏 짜넣었다고 합니다. 그 실용성과 정성이 오늘에까지 이어지고 있습니다.
url: https://kin.naver.com/qna/detail.naver?d1id=11&dirId=11080102&docId=47833655

Example 3:
instruction: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  

토성의 고리는 얼음과 같은 여러 물질로 이루어져 있다고 

In [5]:
# # data
# data = data.map(
#     lambda x:
#     {'text': f"### 명령어: {x['instruction']}\n\n###맥락: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
#     if x['input'] else
#     {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
# )
# data

example_array = list(map(lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }, example_array))

for i in example_array:
  print(i)

data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

{'text': '### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<|endoftext|>'}
{'text': '### 질문: 스웨터의 유래는 어디에서 시작되었나요?\n\n### 답변: 스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 시작되었습니다. 노동자들의 방한복에서 시작된 스웨터는 여가생활과 스포츠의 붐에 힘입어 대중화되었습니다. 이후, 겨울철 이너웨어의 대명사가 되었습니다. 스웨터는 짜서(Knit) 만든 옷을 말하며, 어부들의 방한복으로 짜여졌던 스웨터 중에서도 스코틀랜드 해안지방의 여인들은 바다로 나가는 남편이나 연인, 자식들에게 무사히 돌아올 것을 기원하며 로프나 닻 무늬를 정성껏 짜넣었다고 합니다. 그 실용성과 정성이 오늘에까지 이어지고 있습니다.<|endoftext|>'}
{'text': '### 질문: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  \n\n토성의 고리는 얼음과 같은 여러 물질로 이루어져 있다고 알고 있는데, 카시니가 찍은 사진에서 마치 빛의 띠 처럼 보이는 이유가 무엇인가요? 물질의 공전 속도가 빠르기 때문에 카메라로 담았을 때 빛의 궤적으로 보이는 건가요? 또한, 야간에 빠르게 움직이는 자동차를 장노출로 찍었을 때 빛의 궤적이 생기

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

## 쪼개진 모델 로드

- 원래는 단일 파일이기도 하지만, 작은 파일(약 1GB)로 쪼개서 개별로 로드한 레포를 쓰면 RAM이 터지지 않습니다.

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# 양자화(Quantization)는 모델의 가중치와 활성화 함수의 데이터 표현을 축소하는 기술입니다.

# LLM이 기존에 파인튜닝 방식이 모델 전체를 재학습하는 방식이어서 매우 비효율적
# -> LoRA: LoRA는 기존 pre-trained weights는 frozen해두고 몇개의 dense layer의 weight을 low rank로 분해한 matrices만을 최적화하는 것

# QLoRA: 모델을 16bit에서 4~8bit로 파인튜닝할 수 있게 한다.
# 정보량이 줄어든 만큼 정밀도는 떨어진다 -> NormalFloat(NF)형식으로 하는 것으로 정밀도의 저하를 억제

model_id = "beomi/polyglot-ko-12.8b-safetensors"  # safetensors 컨버팅된 레포
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # 모델을 4비트 정밀도로 메모리에 로드합니다.
    bnb_4bit_use_double_quant=True, #  QLoRa에서 제안하는 이중 양자화를 수행합니다.
    bnb_4bit_quant_type="nf4", # 양자화의 유형입니다. "nf4"는 4비트 NormalFloat를 나타냅니다.
    bnb_4bit_compute_dtype=torch.bfloat16 # 모델을 4비트로 로드하고 저장하지만 필요할 때 일부분을 부분적으로 다양화하고 모든 계산은 16비트 정밀도 (bfloat16)로 수행합니다.
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

## 텍스트 데이터만 tokenize

In [7]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [8]:
data['train'][0]['text']

'### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<|endoftext|>'

In [9]:
# 토큰화 진행됨 -> 'input_ids', 'token_type_ids', 'attention_mask'
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 21155
    })
})

PEFT를 통해 `prepare_model_for_kbit_training`로 Low bit 학습을 준비해줍시다.

In [10]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable() # 모델의 그래디언트 체크포인팅(gradient checkpointing)을 활성화
model = prepare_model_for_kbit_training(model) # Q-LoRA를 위한 셋팅

In [11]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    주어진 모델을 입력으로 받아서 각 파라미터를 순회하며 학습 가능한 파라미터의 개수와 전체 파라미터의 개수를 계산
    """
    trainable_params = 0
    all_param = 0
    # model.named_parameters()를 통해 모델 내의 모든 파라미터와 각 파라미터의 이름을 순회
    for _, param in model.named_parameters():
        # 해당 파라미터의 요소(element) 개수를 반환
        all_param += param.numel()
        # param.requires_grad를 확인하여 해당 파라미터가 학습 가능한지 여부를 판단
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [12]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, # r: Lora 어텐션의 차원 크기를 지정
    lora_alpha=32, # lora_alpha: Lora 스케일링을 위한 알파 파라미터
    target_modules=["query_key_value"], # target_modules: Lora를 적용할 대상 모듈의 이름을 지정
    lora_dropout=0.05, # lora_dropout: Lora 레이어의 드롭아웃 확률
    bias="none", # bias: Lora의 바이어스 유형으로, 'none', 'all', 'lora_only' 중 하나를 선택 가능
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config) # 기존 모델을 LoRA로 파인튜닝을 진행한다.
print_trainable_parameters(model) # 학습 가능한 파라미터의 개수를 출력

# trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424
# 모델이 많은 파라미터를 가지고 있지만, 실제로 학습이 가능한 파라미터는 그 중 일부에 불과하다는 것을 의미 -> 이미 학습된 가중치를 불러와서 하는 것이기 때문.

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


In [13]:
!nvidia-smi

Tue Jul 11 13:30:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    27W /  70W |   9325MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 학습하기!

- 이번 예제에서는 22k개의 아주아주 일부분인 100개 스텝만 학습해봅시다.

In [14]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    # model: 학습할 모델을 지정함
    model=model,
    # train_dataset: 학습에 사용할 데이터셋을 지정
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        # per_device_train_batch_size: 각 장치(디바이스)당 학습 배치 크기를 지정
        per_device_train_batch_size=2,
        # gradient_accumulation_steps: 가중치 축적
        gradient_accumulation_steps=1,
        # max_steps: 최대 학습 스텝을 지정 -> 모델이 최대 몇 번의 스텝을 학습할지를 결정
        max_steps=50,
        # learning_rate: 학습률을 지정 -> 옵티마이저가 모델 파라미터를 업데이트하는 데 사용하는 학습 속도를 나타냄
        learning_rate=1e-4,
        # fp16: FP16 혼합 정밀도 학습을 사용할지 여부를 결정 -> FP16은 반정밀도 부동 소수점 형식을 사용하여 모델을 학습하는 방법 # 음 모르겠음..ㅋ
        fp16=True,
        # logging_steps: 로깅 간격을 지정
        logging_steps=10,
        # output_dir: 학습 결과를 저장할 디렉토리를 지정 -> 학습이 진행되면서 모델 체크포인트, 로깅 파일 등이 해당 디렉토리에 저장됨.
        output_dir="outputs",
        # optim: optimizer 설정
        optim="paged_adamw_8bit"
    ),
    # 언어 모델링을 위해 사용되는 데이터 콜레이터 -> 입력 데이터는 동적으로 배치의 최대 길이에 맞춰 패딩된다.
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.433500
20,2.370900
30,2.139800
40,2.183500
50,1.980100


TrainOutput(global_step=50, training_loss=2.2215558242797853, metrics={'train_runtime': 245.1597, 'train_samples_per_second': 0.408, 'train_steps_per_second': 0.204, 'total_flos': 1064017221672960.0, 'train_loss': 2.2215558242797853, 'epoch': 0.0})

In [15]:
print("wow")

wow


In [16]:
# 모델을 평가 모드로 전환하는 메서드
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [17]:

model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different th

tensor([[    6,     6,     6,  2438,    29,  1832,  4770,   272,    34,   224,
         17081, 26661,   224, 23411,  7743,   224,  7245,    78,  4541,   224]])

In [18]:
def gen(x):
    # 추론한다.
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt', # 파이토치 텐서로 형태로 리턴!
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [19]:
# 뭔소리야;
# def gen(x):
#     q = f"### 질문: {x}\n\n### 답변:"
#     # print(q)
#     gened = model.generate(
#         **tokenizer(
#             q,
#             return_tensors='pt',
#             return_token_type_ids=False
#         ).to('cuda'),
#         max_new_tokens=50,
#         early_stopping=True,
#         do_sample=True,
#         eos_token_id=2,
#     )
#     print(tokenizer.decode(gened[0]))

## 몇 가지 팁

- 만약 학습이 충분히 되지 않으면 `<|endoftext|>` 토큰이 잘 생성되지 않을 수 있습니다
- 이럴떈 충분히 긴 `max_new_tokens`를 준 뒤, `###`으로 잘라서 써보세요. ex) `output.split('###')[0]`
- 아래 결과는 실제 위 학습된(50step, 100개 샘플) 모델의 결과물입니다.
- 생성시에 속도가 꽤 느립니다. 1-2tokens/s 정도라 256토큰 생성시 약 ~3분 시간이 소요됩니다.

In [20]:
gen('건강하게 살기 위한 세 가지 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 첫째, 올바른 식생활을 한다.＇＇건겅한 삶을 위한 식생활을 위한＇ １）４를 ２인한））건겅하게 되는 것은＜되는 것은 건겅하는 것이다..＼하고, 어떤 음식을, ＇하는 것이 올바른 식생활, 음식과 함께 올바른 식생활을 한다.인 것이다.）이다.＜어떤 음식에 맞는 그릇은..그릇을 사용하는 것이 건강에 좋다는 말.음식은 좋은 음식.＜맞는, 그릇.이 맞는,알맞는 그릇과 맞지알맞게.＜그릇을..에는 좋은음식에는 어떤 그릇이 맞을까..어떤 그릇을 쓰고(맞는,그릇을 사용하고),그릇과 맞지 않고,상관없이 먹었습니다.인내한 그릇을,먹고 자고,씻기위한 생활을 하고,상관없이,건강관리에 좋은 음식인을 하는 것이좋은 생활을 위해서는,음식인양,,상관없이 음식과 생활인이기도 하기 위하여 잘하고


`### 질문: 건강하게 살기 위한 세 가지 방법은?`

`### 답변: 첫 번째 방법은 과로하지 않는 것입니다. 두 번째 방법은 물을 많이 마시고, 신선한 음식을 먹는 것입니다. 마지막 방법은 정기적으로 운동을 하는 것입니다. 이러한 세 가지 방법은 규칙적이고 건강하게 사는 데 도움이 됩니다. 하지만 이러한 방법은 누구나 알고 있습니다. 건강한 삶을 살기 위한 다른 방법은 없을까요? 그것은 당신이 가장 만족하지 않는 어떤 것을 추구하는 것 입니다. 예를 들어, 만약 당신이 너무 많이 먹는다고 생각한다면, 당신은 식사량을 줄이는 것이 도움이 될 것입니다. 이러한 연습은 또한 당신의 기분을 더 좋게 해줍니다. 이렇게 당신의 기분이 아주 좋아질 때 당신은 건강한 삶을 살고 있는 것입니다. 이러한 것이 인생에서 가장 만족감을 주는 방식이며, 또한 건강하게 사는 방법입니다. 당신의 건강과 행복을 유지하는 중요한 단계는 어떤 것이 있습니까? 이러한 것을 알아내기 위해 다음 질문에 대해 생각해 보십시오. 그리고 나서 그것들을 적고 목록을 만들어 보십시오. 이 과정은 무엇을 생각하고`

In [21]:
gen('슈카월드가 무엇인가요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 슈카월드가 무엇인가요?

### 답변: 슈카월드는 유튜브 채널에서 나온 것이며, 채널이 가지는 의미로는, 채널 이름입니다.https://www.youtube.threaelecratiiieviorie.(meaninglessgae etoile http:https://lee121jae.tistory.webmeanlee) https://line7l23sisamembreacrati20이라는 웹사이트 이름으로도 사용되었으며,bridlleavl2rairielle0981, aibrgae: http:http:https://twitter.com/#thetwenthremehttp://naver.me/xArLo00:19:00:00|aasphttp:1/?thelee20:00:00:25:00:40:40|tel0701,:00:20:00:30 athred:00:00:40:40|


`### 질문: 슈카월드가 무엇인가요?`

`### 답변: 슈카월드는 유튜브 채널이며, 다양한 경제 분야의 주제를 다루고 있습니다. 경제분야에 관심이 있으신 분들은 한번쯤은 들어보셨을 채널일 것입니다. 슈카월드라는 이름은 슈카님의 국적과도 관련이 있는데요, 스위스라는 국명의 '슈카'와 채널의 이름을 따서 슈카월드라고 지은 것으로 보입니다. 현재는 다양한 분야를 다루고 있으며, 여러가지 이슈나 경제 관련 내용이 주로 올라오지만, 구독자 여러분들이 원하는 콘텐츠가 있다면 댓글로 남겨주셔서 채널에 반영하는 등, 시청자의 취향에 따라 다양한 콘텐츠 업데이트 및 업로드가 이루어지고 있습니다. � 인준: 슈카월드는 현재 구독자수 120만 명 이상(!), 조회수는 6600만 뷰 이상을 기록하고 있습니다. 구독자와 조회수의 차이는 컨텐츠를 얼마나 즐겨보는지와 관계가 있는 것으로 보입니다. 슈카월드는 경제관련 내용 이외에도, 여행, 정치, 문화, 역사, 스포츠, 상식, 정보, 시사 등 다양한 분야에 대한 상식, 정보를 다루고 있는 채널입니다. 따라서, 슈카월드를 구독한 많은 분들은`


In [22]:
gen('공공성을 측정하기위한 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 공공성을 측정하기위한 방법은?

### 답변: 공공성을 측정하기위한 몇 가지 방법 중에서도 사용하고 있는 가장 대표적으로 사용되는 방법은 다음과 같이 세 가지다/도(가장)/.이를테면, 개인적인 차원에서는 사회문화적으로 가장 많이 사용하는 방법이 개인적인 영역, 제도적인차원에서는 이에 따라 다를 수 있고, 실제 적용시 이 방법의 장점과 단점이 있으므로, 가장 바람직한 영역에서 활용해보고자한다.제도적인 차원에서는, 실제 활동들을 반영하는, 개인적인 차원에서 개인의 인식과 관련된 활동영역에서는, 이러한 방법에서는 사람들의 인식적인 영역에서는, 방법, 제도적인차원에서, 사용될 수적인적인 차원에서는,실제는 제도적인 문제들을 해결하는 과정으로, 현재, 사회적, 문화적인 차원에서, 제도적인 과정들은,실제는 가장 핵심적인 요소는서는 가장 취약한 영역으로서, 사람들의 실제 활동은 의식의 변화를 가져오게 되고,제도의 경우는 문제해결(예방,치유,인식,관심 등)에 대한 과정 변화과정의 문제라고도 한다.


`### 질문: 공공성을 측정하기위한 방법은?`

`### 답변: 공공성을 측정하기위한 몇 가지 방법 중 하나는 공공기관의 서비스에 대한 청구 비용을 측정하는 것입니다. 이를 위해 A/B 테스트가 사용됩니다. 이 방법은 청구 비용이 평균 값 A보다 클경우 A의 사용을 요구하고, 청구 비용이 평균 값 A보다 작다면 B의 사용을 요구합니다. 공공 기관의 실제 비용을 측정해 공공성을 측정하는 다른 방법에는 정부에서 지불하는 재정 예산이 있습니다. 이러한 비용은 재정적 지출과 관련한 정보를 공개하는 것이 적절합니다. 이러한 정보는 또한 공공 서비스에 대한 청구 비용 측정에서도 동일하게 적용될 수 있습니다. 이를 통해, 공공기관의 공공성이 측정될 수 있습니다.### 질문: 특정 공공 서비스에 대한 청구 비용이 적절한 경우는?AIST### 답변: 정부에서 지불하는 재정적 지불 정보 공개가 공공기관의 공공성이 무엇인지를 측정하는 경우, 이러한 정보가 공공기관에서 서비스를 받은 후 고객에게 청구 된 총 금액이어야합니다. 만약, 지불 금액을 서비스 비용과 같은 개념으로 이해한다면, 공공기관의 다른 서비스 비용 또한 청구 비용으로 측정`

In [23]:
gen('주식 시장에서 안정적으로 수익을 얻기 위한 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 주식 시장에서 안정적으로 수익을 얻기 위한 방법은?

### 답변: 주식시장은 변동성이 높아지면 낮은 변동성을 갖는 정보를 찾는 습관이 있는 투자자의 투자수익을 올릴 수눼釉끈다±他의 투자수익은 매우 높은 정보를 얻을 수눌리딩에 대한 투자정보이다.여벌에 대한 만족도를 높일 수殆등한정보의 비대칭성으로 인한 불만족과 관련하여 다양한 의견이 있을 수 있으나, 실제로 주식에 대해 체계적 이해력높은 분들은 안정적으로 높은 투자수익을 높이는 기본적으로 이해하기 어려운 문제해결 방법에 대해 체계적으로인덱스 펀드 도입이 가능하나, 일반 투자가인 우리에게 큰 효용가치가 없는 항목으로 평가절하가 현저히 다른 시장에서는 투자가 큰 시장이 안정적으로 성장성이 높다고 합니다수가 판단할 수에 대한 신뢰성 있는 정보는 부족을 의미, 즉 큰 시장의한 큰 어려움은 없었으니 신뢰감을 가질 수 있습니다.펀드를 이용하여 시장성을 높이는 노력에 관한 문제 해결이때까지의 정보를 바탕으로 높아진 신뢰성과 효율성을 추구할 수 있습니다.펀드 규모는리딩시에 관한 내용에 대해서는 시장


`### 질문: 주식 시장에서 안정적으로 수익을 얻기 위한 방법은?`

`### 답변: 주식시장은 변동성이 높습니다. 하지만, 변동성이 높음에도 불구하고 수익을 얻으려면 장기적으로 봐야하고, 기업의 미래 가치를 분석하고 투자해야 합니다. 기업의 주식을 샀다면 그 기업의 미래 가치가 좋아야 하고 성장해야 합니다. 그래야 꾸준히 높은 주가 상승을 볼 수 있습니다. 하지만, 단기간의 수익을 얻길 원한다면 다른 방법을 권합니다. 이러한 이유로 주식 투자를 하는 많은 개인들은 단타의 형태를 보입니다. 단타는 단기적으로 수익을 얻고 빠져나오는 방법을 의미합니다. 주식을 사고 일정 기간이 지나지 않았음에도 자신의 수익이 났다면 그 수익금을 챙기고 나오는 형태입니다. 단타는 변동성이 높은 주식 시장에서 비교적 짧은 기간 동안 수익을 보고 나오는 방법입니다. 기업에서 꾸준히 수익을 내는 것보다 훨씬 쉬운 만큼 리스크가 큰 방법입니다. 때문에 개인이 주식 투자에서 안정적으로 수익을 얻길 원하신다면 장기적으로 투자를 하시는 것을 권합니다. 꾸준한 수익을 내기 위해 노력하는 기업을 찾으시고 함께 성장하면 됩니다. 이러한 노력은 시간이 걸립니다. 단, 기간적인 부분에서 시간이 오래 걸릴`


In [24]:
gen('풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?

### 답변: 풋옵션은ggrsbspmhover)포지션)'호출'As'+BbitiiRCIIiTbTPLAzFbIiTbTdGTalBbTGTPPdBaAASeIBAAWGPlBrBBCrBaBbItIttPlanAmEmeAnimErAnimErgoSErAsiaRulezHTML5VertClaro AniTbFTrAttrcAsiaNoLisianGTGTMnIngSelectHTML5NANOHonAPPHTMUSelectRuleZiMhOperanTrickOfLeAnAPPLEPCombinedAPPLEdPLAwPoliticsZiChEdIottPaUnoAPPLECombinedStartMeanwellToLookingAsiaBarTexASimTurkMeaningSelect


`### 질문: 풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?`

`### 답변: 풋옵션과 콜옵션의 차이점은, 우선 둘 다 금융 시장에서 거래가 가능한 파생상품이며 이 옵션들은 거래 방법이 다릅니다. 주식을 사고파는 주식거래와 달리, 이 상품은 당사자 사이에 계약을 통해 거래하게 됩니다. 또한, 주식과 달리, 두 옵션상품 모두 시장 상황에 따른 가격의 상승과 하락에 영향을 받을 수 있습니다. 그리고 두 옵션의 차이점과 더 중요한 차이점은, 옵션은 거래 금액이 고정되지 않습니다. 즉, 주식거래와 같이 계약한 금액에 고정된 금액으로 거래하는 것이 아니라는 의미입니다. 이는 주가가 급변할 수도, 가격 변동 폭이 매우 클 수도 있는 상황에서 주식 거래와 옵션 거래를 비교했을 때, 옵션 거래가 훨씬 더 위험한 이유이기도 합니다. 그래서 일반 개미 투자자는 주식거래나 선물 거래와 같이 일정한 금액이 고정되어 거래되는, 안정성이 더 큰 일반적인 주식거래를 합니다. 일반 개미 투자자가 이러한 주식거래의 일종인 선물과 주식거래의 일종인 옵션거래에서 선택해야 하는 포지션은, 우선 선물거래는 선물 매도 포지션 (자신의 보유 주식을 선물 매도 계약에 포함하여 보유`


In [25]:
gen('풋옵션 매도와 콜옵션 매수의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 풋옵션 매도와 콜옵션 매수의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?

### 답변: 풋옵션을 매수한 의미는 미리 정해진 시점의 지정된 수준의의더 많은 증거의 증거의영역에서 벗어났다.여기서의 각 단계의 차이점 중가장 적합한 수준에서가가장 많은 부분을차지하는 각 단계별 증거의부분집약에서증거를 사용하지않은 상태의부분적합하게하부에대한이해하는어사용수준이다.획정시나리오이다표방을참고로이성적인판단이어디까지만일반인의감이라면이성적판단은판단도는수치심도그부분과 관련된 사항이다, 라는 가정에서는 가장 많이 사용되는시기에 있어, 그 분야의 다른 판단으로판단되어, 가장많이 사용되는나아가야 하는 가정에서 차이가 나게 되며, 일반적으로 전문가, 또는 일반화과정에서, 또는 일반적인 분야일 경우를 말해주는 판단력과는 또 다른 기준은도 있는데, 이러한 전문가는 그 분야에 전문용어로 표현되기 때문에, 여기서 표현되는 수준이 어느 정도는 이해력과 판단력이 있을 때로, 필요한 정보를 판단하고 있으며, 특히 중요한 것은


`### 질문: 풋옵션 매도와 콜옵션 매수의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?`

`### 답변: 풋옵션과 콜옵션의 매수와 매도 포지션의 정의를 먼저 알아보겠습니다. 풋옵션이란 '풋'이란 판매하는 사람이 파는 금액을 의미하고, '옵션'이란 선택 사항을 의미합니다. 즉, 풋옵션은 '풋'이라고 표기를 해야 하고, 선택 사항이 있음을 의미합니다. 이러한 옵션은 주가에 대해서 행사하는 권리를 부여하는 것으로, 이러한 옵션을 선택하였을 때는 해당 주식과 같은 방향으로 매도하는 것과 구매하는 것이 가능합니다. 예를 들어서, 주가가 하락할 것이라는 기대가 있다면, 이러한 예상으로 인하여 주가가 떨어질 것이기 때문에 일반 매도 포지션으로 주가의 하락에 참가하는 것이 낫지만, 주가가 하락할 때 해당 주식을 일정 부분 상승의 방향으로, 또는 상승의 폭만큼을 보장받고 싶다면, 이러한 옵션을 선택하는 것입니다. 즉, 옵션에서 주가가 상승을 할 때는 콜옵션으로 선택하는 것이고, 주가가 하락하게 될 때는 풋옵션을 선택하게 됩니다. 이러한 선택을 할 경우, 일반적으로 주가의 하락에 대해서 상승의 방향으로 상승폭`


In [26]:
gen("마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?

### 답변: 마진콜이 발생하지 않기 위해서는 여러 이유가 있는데, 우선, 다른 무엇이 있어야 한다.bit.hr...다른 무엇보다도, 여러 가지가 있어야 하는 것은 우선권이나, 어떤 것이 있을 때, 어떤 이유의 제약 사항이 발생하며," 이는 또한 어떤 요소들이 있는 내용이 있습니다. 우선적인 요소가 어떤 것들을 ""어떤 방법으로 정의하느냐 마키는 어떤 경우에 속하고", 우선적이며, 또한 여러 가지로 정의되어 있습니다. 즉," ""어떤 형태로 조합하는 것""들이 있고", 이는 또한 여러 가지로 정의되어 있는 경우도 있고, 어떤 것은, 어떤 내용에, 특정한 것은, 특정한 것으로," 또한 ""어떤 것이 있습니다. 둘째는 ""다른 한편"" 즉 ""또한 어떤 상황의 정의해 주시고"," 다른 여러 가지의 형태의," 또는 여러 가지의 형식적 구조로 정의할 내용의 수 있고 그것은 또한 어떤 경우는 다른 사람이 정의하고 있습니다. ""즉", 그 이유는 여러 가지 요소들은,,


`### 질문: 마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?`

`### 답변: 마진콜이 발생하는 이유는 은행이 고객에게 지급하고자 하는 금액이 발생한 금액보다 많은 경우, 추가로 내야 하는 금액을 계산해서 계좌에서 더 많은 금액을 인출하게 되는 것입니다. 이렇게 되면, 계좌를 관리하는 은행 입장에서는 계좌에서 추가로 꺼내야 하기 때문에, 은행의 신뢰도가 땅으로 떨어지고, 계좌를 사용하는 은행 고객도 추가로 내야 하는 금액에 대한 불안감을 느낄 수 있습니다. 이런 불안감에 대해 잘 보여주는 사례가 영화 마진콜입니다. 따라서, 이런 현상을 막기 위해서는 고객에게 먼저 지급할 금액과 인출하는 금액을 정확하게 파악하는 것이 중요합니다.`



## 모델 저장 & 업로드

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 

In [ ]:
# model.push_to_hub('beomi/qlora-koalpaca-polyglot-12.8b-50step')
# 아래 아이디 부분을 수정해서 쓰세요.
model.push_to_hub('허깅페이스아이디/qlora-koalpaca-polyglot-12.8b-50step')

- 위 예시코드는 https://huggingface.co/beomi/qlora-koalpaca-polyglot-12.8b-50step 에 올라갑니다. 사용하시는 이름으로 바꿔쓰세요.

# Hugginface Hub에 올린 LoRA Weight 저장한것 불러오는 튜토리얼 코드

- HF에 업로드 후, 실제 다운받아 사용시 쓰는 코드 예제입니다.

--> https://github.com/Beomi/KoAlpaca/blob/main/2023_06_08_PEFT%EB%A1%9CLoRA%EB%A1%9C%EB%93%9C.ipynb